In [ ]:
# Paso 1: Configuración del Entorno
import os
import sys
import datetime

print(f"Fecha de ejecución: {datetime.datetime.now()}")
print(f"Python version: {sys.version}")

try:
    import ultralytics
except ImportError:
    print("Instalando Ultralytics YOLOv8...")
    !pip install ultralytics -q
    import ultralytics

from ultralytics import YOLO
import yaml
import shutil

print(f"Ultralytics version: {ultralytics.__version__}")

import torch
if torch.cuda.is_available():
    print(f"GPU disponible: {torch.cuda.get_device_name(0)}")
else:
    print("GPU no disponible, se usará CPU.")

print("✅ Entorno configurado.")

# Rutas del dataset y extracción
zip_path = '/content/dataset_maestro_danos.zip'  # <-- Cambia aquí si tu archivo tiene otro nombre
extract_path = '/content/dataset_damage'

In [ ]:
# Paso 2: Descomprimir el Dataset
import shutil as sh
total, used, free = sh.disk_usage("/")
print(f"Espacio libre: {free // (2**30)} GB")

if not os.path.exists(zip_path):
    raise FileNotFoundError(f"No se encontró el archivo .zip en la ruta: {zip_path}")

if not zip_path.endswith('.zip'):
    raise ValueError("El archivo debe ser un .zip")

print(f"Descomprimiendo {zip_path} en {extract_path} ...")
!unzip -q "{zip_path}" -d "{extract_path}"
print(f"✅ Dataset descomprimido en: {extract_path}")

In [ ]:
# Paso 3: Localizar y validar el archivo data.yaml
import yaml
import glob
from collections import Counter

# Busca el archivo data.yaml en el directorio de extracción
data_yaml_path = None
for root, dirs, files in os.walk(extract_path):
    if 'data.yaml' in files:
        data_yaml_path = os.path.join(root, 'data.yaml')
        break

if not data_yaml_path:
    raise FileNotFoundError("No se encontró el archivo data.yaml en el dataset descomprimido.")

with open(data_yaml_path, 'r') as file:
    data_yaml_content = yaml.safe_load(file)
    print(f"Archivo data.yaml encontrado en: {data_yaml_path}")
    print("Clases detectadas:", data_yaml_content.get('names', 'No se encontraron clases'))
    assert 'train' in data_yaml_content and 'val' in data_yaml_content, "data.yaml debe tener rutas 'train' y 'val'"

# Corrige rutas de train y val para que sean absolutas
dataset_root_dir = os.path.dirname(data_yaml_path)
for split in ['train', 'val', 'test']:
    if split in data_yaml_content:
        original_path = data_yaml_content[split]
        absolute_path = os.path.join(dataset_root_dir, original_path) if not os.path.isabs(original_path) else original_path
        data_yaml_content[split] = os.path.normpath(absolute_path)

# Guarda el YAML corregido
with open(data_yaml_path, 'w') as file:
    yaml.dump(data_yaml_content, file, default_flow_style=False)

print("Rutas corregidas en data.yaml:")
print(yaml.dump(data_yaml_content, default_flow_style=False))

# Análisis de distribución de clases
label_dir = data_yaml_content['train'].replace('images', 'labels')
label_files = glob.glob(os.path.join(label_dir, '*.txt'))
class_counts = Counter()
for lbl in label_files:
    with open(lbl) as f:
        for line in f:
            class_id = line.strip().split()[0]
            class_counts[class_id] += 1
print("Distribución de clases en entrenamiento:", dict(class_counts))

train_images = glob.glob(os.path.join(data_yaml_content['train'], '*.*'))
val_images = glob.glob(os.path.join(data_yaml_content['val'], '*.*'))
print(f"Número de imágenes de entrenamiento encontradas: {len(train_images)}")
print(f"Número de imágenes de validación encontradas: {len(val_images)}")

In [ ]:
# Paso 4: Visualización rápida del dataset
import matplotlib.pyplot as plt
from PIL import Image

# Selecciona las primeras 5 imágenes de entrenamiento
img_files = glob.glob(os.path.join(data_yaml_content['train'], '*.jpg'))[:5]

# Verifica si todas tienen su archivo de etiqueta correspondiente
missing_labels = []
for img_path in img_files:
    lbl_path = img_path.replace('images', 'labels').replace('.jpg', '.txt')
    if not os.path.exists(lbl_path):
        missing_labels.append(img_path)

if missing_labels:
    print("Imágenes sin etiqueta:", missing_labels)
else:
    print("Todas las imágenes seleccionadas tienen etiqueta.")

if img_files:
    print(f"Mostrando {len(img_files)} imágenes de entrenamiento:")
    plt.figure(figsize=(15, 3))
    for i, img_path in enumerate(img_files):
        img = Image.open(img_path)
        plt.subplot(1, len(img_files), i + 1)
        plt.imshow(img)
        plt.axis('off')
        plt.title(os.path.basename(img_path))
    plt.show()
else:
    print("No se encontraron imágenes de entrenamiento para mostrar.")

In [ ]:
# Paso 5: Entrenamiento del modelo YOLOv8 para daños
model_type = 'yolov8n.pt'  # Cambia a yolov8s.pt, yolov8m.pt, etc. si tienes más recursos
model = YOLO(model_type)

print("\n🚀 Iniciando entrenamiento para detección de daños...")
try:
    results = model.train(
        data=data_yaml_path,
        epochs=150,
        imgsz=640,
        project='TFM_Models_Damage',
        name='Damage_Detector',
        batch=16,
        patience=20,
        optimizer='auto',
        augment=True
    )
    print("\n✅ Entrenamiento de daños completado.")
except Exception as e:
    print(f"❌ Error durante el entrenamiento: {e}")

In [ ]:
# Paso 6: Reanudar un Entrenamiento Interrumpido
import glob
import os
from ultralytics import YOLO

# Usa la ruta correcta para el detector de daños
run_dir = '/content/TFM_Models_Damage/Damage_Detector*'

try:
    run_folders = sorted(glob.glob(run_dir))
    if not run_folders:
        print("No se encontró ninguna carpeta de entrenamiento previa.")
    else:
        latest_run_dir = run_folders[-1]
        last_checkpoint_path = os.path.join(latest_run_dir, 'weights/last.pt')

        if os.path.exists(last_checkpoint_path):
            print(f"Punto de control encontrado en: {last_checkpoint_path}")
            print("Reanudando el entrenamiento...")
            model = YOLO(last_checkpoint_path)
            results = model.train(resume=True)
            print("Entrenamiento reanudado y completado.")
        else:
            print(f"No se encontró el archivo 'last.pt' en la carpeta: {latest_run_dir}")
except Exception as e:
    print(f"Error al intentar reanudar el entrenamiento: {e}")

In [ ]:
# Paso 7: Evaluación y Visualización de Resultados
print("📊 Resultados de entrenamiento:")
metrics = results.results_dict if hasattr(results, 'results_dict') else results
print(metrics)

# Visualiza y guarda predicciones en imágenes de validación
val_imgs = glob.glob(os.path.join(data_yaml_content['val'], '*.jpg'))[:3]
if not val_imgs:
    print("⚠️ No se encontraron imágenes de validación para mostrar predicciones.")
else:
    for img_path in val_imgs:
        pred = model.predict(img_path, conf=0.25, save=True)
        pred[0].show()
        pred[0].save(f"pred_{os.path.basename(img_path)}")
        print(f"Predicción guardada para {img_path}")

In [ ]:
# Paso 8: Exportar y Guardar el Modelo Entrenado
best_model_path = '/content/TFM_Models_Damage/Damage_Detector/weights/best.pt'
final_model_name = 'damage_detector_model.pt'

if os.path.exists(best_model_path):
    shutil.copy(best_model_path, f"/content/{final_model_name}")
    print(f"\n🎉 ¡Modelo final guardado como '{final_model_name}' en la carpeta principal de Colab!")
    print("Puedes descargarlo desde el panel de archivos a la izquierda.")
    exported_onnx = model.export(format='onnx')
    print(f"Modelo exportado a ONNX en: {exported_onnx}")
    exported_torchscript = model.export(format='torchscript')
    print(f"Modelo exportado a TorchScript en: {exported_torchscript}")
    print(f"Tamaño del modelo final: {os.path.getsize(best_model_path) / (1024*1024):.2f} MB")
else:
    print(f"❌ ERROR: No se encontró el archivo del modelo entrenado en {best_model_path}")

In [ ]:
# Paso final: Visualización avanzada de métricas y recomendaciones

import matplotlib.pyplot as plt

# Mostrar mAP global y por clase si está disponible
if hasattr(results, 'metrics'):
    metrics_data = results.metrics
    if 'map50' in metrics_data:
        print(f"mAP@0.5 global: {metrics_data['map50']:.3f}")
    if 'map50-95' in metrics_data:
        print(f"mAP@0.5:0.95 global: {metrics_data['map50-95']:.3f}")
    if 'map_per_class' in metrics_data and 'names' in metrics_data:
        print("\n--- mAP por clase ---")
        for i, name in enumerate(metrics_data['names']):
            print(f"{name}: {metrics_data['map_per_class'][i]:.3f}")
        # Gráfica de mAP por clase
        plt.figure(figsize=(10,4))
        plt.bar(metrics_data['names'], metrics_data['map_per_class'])
        plt.title('mAP por clase')
        plt.ylabel('mAP')
        plt.xticks(rotation=45)
        plt.show()

# Matriz de confusión si está disponible
if hasattr(results, 'confusion_matrix'):
    cm = results.confusion_matrix
    plt.figure(figsize=(8,6))
    plt.imshow(cm, cmap='Blues')
    plt.title('Matriz de confusión')
    plt.xlabel('Predicción')
    plt.ylabel('Real')
    plt.colorbar()
    plt.show()

# Ejemplo de visualización de predicciones difíciles (opcional)
val_imgs = glob.glob(os.path.join(data_yaml_content['val'], '*.jpg'))[:3]
if val_imgs:
    print("\nEjemplos de predicciones en imágenes de validación:")
    for img_path in val_imgs:
        pred = model.predict(img_path, conf=0.25)
        pred[0].show()

# Recomendación profesional
print("\n✅ Revisión final completada.")
print("- Analiza las clases con menor mAP para mejorar el dataset o ajustar hiperparámetros.")
print("- Usa la matriz de confusión para identificar errores frecuentes entre clases.")
print("- Guarda estos gráficos y métricas para tu informe o documentación.")